# Chapter 25: Smart Contract Testing

---

Testing is not just a good practice in smart contract development—it's an absolute necessity. Unlike traditional software, where bugs can be patched with a hotfix, smart contracts are often immutable and control real assets. A single vulnerability can lead to catastrophic financial loss, as seen in numerous high-profile hacks. This chapter explores the testing pyramid for smart contracts, from unit tests to advanced techniques like fuzzing and formal verification, using industry-standard tools like Hardhat, Foundry, and Echidna. By the end, you'll be equipped to write comprehensive test suites that catch bugs before they reach production.

---

## 25.1 Testing Philosophy

### 25.1.1 Why Testing is Critical

Consider the stakes: a smart contract may hold millions of dollars, and its code cannot be changed after deployment (unless upgradeable, but even then upgrades are risky). Traditional software can rely on post-deployment patches; blockchain cannot. Therefore, testing must be exhaustive.

**Consequences of inadequate testing:**
- **The DAO hack (2016)**: Reentrancy vulnerability led to loss of 3.6M ETH.
- **Parity Wallet hack (2017)**: $30M frozen due to library contract self-destruct.
- **Numerous DeFi exploits**: Flash loan attacks, oracle manipulation, etc.

**Testing goals:**
- Verify functional correctness (does the contract do what it's supposed to?).
- Ensure security properties (no reentrancy, access control, etc.).
- Measure gas usage (optimize to reduce costs).
- Test edge cases (zero amounts, extreme values, unexpected callbacks).

### 25.1.2 Testing Pyramid

The testing pyramid for smart contracts mirrors traditional software but with blockchain-specific layers:

```
Testing Pyramid:

          /\
         /  \      End-to-End / Integration
        /    \     (Full protocol interactions)
       /------\
      /        \   Integration
     /          \  (Contract-to-contract)
    /------------\
   /              \ Unit Tests
  /________________\ (Individual functions)
  
         /\
        /  \      Fuzzing / Property-based
       /____\     (Randomized inputs)
```

- **Unit tests**: Test individual functions in isolation, mocking dependencies.
- **Integration tests**: Test interactions between contracts (e.g., a token and a DEX).
- **End-to-end tests**: Test the entire system, including frontend (rare in pure Solidity, often in JS).
- **Fuzzing**: Automatically generate random inputs to find unexpected behavior.
- **Formal verification**: Mathematically prove properties (advanced, used for critical systems).

---

## 25.2 Unit Testing

Unit tests verify the smallest testable parts of a contract—usually individual functions. They are fast, isolated, and form the foundation of the testing pyramid.

### 25.2.1 Testing with Hardhat

Hardhat uses Mocha and Chai for testing. Tests are written in JavaScript/TypeScript and run against a local Hardhat Network.

**Setup:**
```bash
npm install --save-dev @nomicfoundation/hardhat-toolbox
```

**Example test for a simple Token contract:**

```javascript
const { expect } = require("chai");
const { ethers } = require("hardhat");

describe("Token", function () {
  let Token, token, owner, addr1, addr2;

  beforeEach(async function () {
    // Get the ContractFactory and Signers here.
    Token = await ethers.getContractFactory("Token");
    [owner, addr1, addr2] = await ethers.getSigners();

    // Deploy contract with initial supply 1000
    token = await Token.deploy(1000);
  });

  describe("Deployment", function () {
    it("Should set the right owner", async function () {
      expect(await token.owner()).to.equal(owner.address);
    });

    it("Should assign the total supply of tokens to the owner", async function () {
      const ownerBalance = await token.balanceOf(owner.address);
      expect(await token.totalSupply()).to.equal(ownerBalance);
    });
  });

  describe("Transactions", function () {
    it("Should transfer tokens between accounts", async function () {
      // Transfer 50 tokens from owner to addr1
      await token.transfer(addr1.address, 50);
      expect(await token.balanceOf(addr1.address)).to.equal(50);

      // Transfer 50 tokens from addr1 to addr2
      await token.connect(addr1).transfer(addr2.address, 50);
      expect(await token.balanceOf(addr2.address)).to.equal(50);
    });

    it("Should fail if sender doesn't have enough tokens", async function () {
      const initialOwnerBalance = await token.balanceOf(owner.address);

      // Try to send 1 token from addr1 (0 balance) to owner
      await expect(
        token.connect(addr1).transfer(owner.address, 1)
      ).to.be.revertedWith("ERC20: transfer amount exceeds balance");

      // Owner balance should not have changed
      expect(await token.balanceOf(owner.address)).to.equal(initialOwnerBalance);
    });

    it("Should update balances after transfers", async function () {
      const initialOwnerBalance = await token.balanceOf(owner.address);

      // Transfer 100 tokens from owner to addr1
      await token.transfer(addr1.address, 100);

      // Transfer another 50 from owner to addr2
      await token.transfer(addr2.address, 50);

      // Check balances
      expect(await token.balanceOf(owner.address)).to.equal(initialOwnerBalance - 150);
      expect(await token.balanceOf(addr1.address)).to.equal(100);
      expect(await token.balanceOf(addr2.address)).to.equal(50);
    });
  });
});
```

**Key Hardhat testing features:**
- `ethers.getSigners()`: Get accounts for testing.
- `token.connect(addr1)`: Impersonate another account.
- `expect().to.be.revertedWith(...)`: Check for specific revert reasons.
- `await` for asynchronous contract calls.

### 25.2.2 Testing with Foundry

Foundry allows writing tests in Solidity itself, which can be faster and more expressive for complex contract interactions. Tests are contracts that inherit from `Test` (from forge-std).

**Example Foundry test:**
```solidity
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.19;

import "forge-std/Test.sol";
import "../src/Token.sol";

contract TokenTest is Test {
    Token public token;
    address owner = address(1);
    address user = address(2);

    function setUp() public {
        vm.prank(owner);
        token = new Token(1000 ether);
    }

    function testDeployment() public {
        assertEq(token.owner(), owner);
        assertEq(token.totalSupply(), 1000 ether);
        assertEq(token.balanceOf(owner), 1000 ether);
    }

    function testTransfer() public {
        vm.prank(owner);
        token.transfer(user, 100 ether);

        assertEq(token.balanceOf(owner), 900 ether);
        assertEq(token.balanceOf(user), 100 ether);
    }

    function testFailTransferInsufficientBalance() public {
        vm.prank(user); // user has 0 tokens
        token.transfer(owner, 1 ether);
        // Should revert
    }

    // Fuzzing test (random inputs)
    function testFuzzTransfer(uint256 amount) public {
        vm.assume(amount <= 1000 ether);
        vm.prank(owner);
        token.transfer(user, amount);

        assertEq(token.balanceOf(owner), 1000 ether - amount);
        assertEq(token.balanceOf(user), amount);
    }
}
```

**Cheatcodes:** Foundry provides cheatcodes (e.g., `vm.prank`, `vm.warp`, `vm.roll`) to manipulate the EVM state, which is extremely powerful.

### 25.2.3 Assertion Libraries

- **Chai matchers (Hardhat)**: `expect(await token.balanceOf(addr1)).to.equal(100)`
- **Forge std (Foundry)**: `assertEq`, `assertTrue`, `assertGt`, etc.
- **OpenZeppelin Test Helpers**: `expectEvent`, `expectRevert`, etc.

### 25.2.4 Test Organization

Organize tests by contract and functionality. Use `describe` blocks to group related tests. Keep tests independent; each test should set up its own state (using `beforeEach` in Hardhat, `setUp` in Foundry).

**Naming convention:** `test_<function>_<scenario>_<expected>`.

---

## 25.3 Integration Testing

Integration tests verify interactions between multiple contracts, often mimicking real-world scenarios.

**Example: Testing a token and a DEX pair.**

```javascript
// Hardhat test
describe("DEX and Token Integration", function () {
  let token, dex, owner, user;

  beforeEach(async () => {
    [owner, user] = await ethers.getSigners();
    const Token = await ethers.getContractFactory("Token");
    token = await Token.deploy(1000000);
    const DEX = await ethers.getContractFactory("SimpleDEX");
    dex = await DEX.deploy(token.address);
  });

  it("Should allow swapping tokens for ETH", async () => {
    // Owner provides liquidity
    await token.approve(dex.address, 1000);
    await dex.addLiquidity(1000, { value: ethers.parseEther("1") });

    // User swaps
    await token.connect(user).approve(dex.address, 100);
    await dex.connect(user).swapTokenForEth(100);
    // Check balances...
  });
});
```

Integration tests are slower but catch issues that unit tests miss (e.g., interaction order, approval issues).

---

## 25.4 Fuzzing and Property-Based Testing

Fuzzing automatically generates random inputs to test invariants. It's excellent for finding edge cases that manual tests overlook.

### 25.4.1 What is Fuzzing?

Fuzzing feeds random data into functions and checks that certain properties hold. For example, "the sum of balances should always equal total supply."

### 25.4.2 Foundry Fuzz Tests

Foundry has built-in fuzzing. Any test function with parameters becomes a fuzz test.

```solidity
function testFuzz_Transfer(uint256 amount) public {
    vm.assume(amount <= token.balanceOf(owner));
    vm.prank(owner);
    token.transfer(user, amount);
    assertEq(token.balanceOf(owner) + token.balanceOf(user), 1000 ether);
}
```

You can bound inputs with `vm.assume`. Foundry runs many iterations (default 256, configurable).

### 25.4.3 Echidna

Echidna is a more advanced fuzzer for Solidity. It allows writing property tests in Solidity and runs them with sophisticated coverage-guided fuzzing.

**Installation:**
```bash
cargo install echidna
```

**Example property (Echidna):**
```solidity
contract TestToken is Token {
    function echidna_test_balance_under_total() public view returns (bool) {
        return totalSupply() >= balanceOf(msg.sender);
    }
}
```

Run:
```bash
echidna-test . --contract TestToken
```

Echidna will try to find counterexamples where the property fails.

---

## 25.5 Test Coverage

Test coverage measures how much of your code is exercised by tests. It helps identify untested paths.

### 25.5.1 Measuring Coverage

**Hardhat** with `solidity-coverage`:
```bash
npm install --save-dev solidity-coverage
```

Add to `hardhat.config.js`:
```javascript
require("solidity-coverage");
```

Run:
```bash
npx hardhat coverage
```

**Foundry** has built-in coverage:
```bash
forge coverage
```

It generates an HTML report showing which lines, branches, and functions are covered.

### 25.5.2 Coverage Tools

- **lcov** format reports can be viewed with tools like `genhtml`.
- Continuous integration can enforce minimum coverage thresholds.

**Target:** Aim for >95% coverage, but remember 100% coverage doesn't mean 100% bug-free; it only measures executed lines.

---

## 25.6 Fork Testing

Fork testing allows you to test against a snapshot of a live network (e.g., Ethereum mainnet). This is invaluable for testing interactions with existing protocols (Uniswap, Aave, etc.) without risking real funds.

### 25.6.1 Mainnet Forking

**Hardhat** can fork mainnet:
```javascript
module.exports = {
  networks: {
    hardhat: {
      forking: {
        url: "https://eth-mainnet.g.alchemy.com/v2/YOUR_API_KEY",
        blockNumber: 15000000 // optional: pin to a block
      }
    }
  }
};
```

Now in your tests, you have access to all mainnet state.

**Example: Testing a flash loan with Aave on a fork:**
```javascript
describe("Flash Loan", function () {
  it("Should execute a flash loan", async function () {
    // Use mainnet addresses
    const aaveLendingPool = "0x7d2768dE32b0b80b7a3454c06BdAc94A69DDc7A9";
    const dai = "0x6B175474E89094C44Da98b954EedeAC495271d0F";

    // Impersonate a whale account that has DAI
    await hre.network.provider.request({
      method: "hardhat_impersonateAccount",
      params: ["0x..."]
    });
    const whale = await ethers.getSigner("0x...");
    // ... perform flash loan
  });
});
```

### 25.6.2 Testing with Real Protocol States

Fork testing enables:
- Checking how your contract interacts with deployed protocols.
- Simulating complex scenarios (e.g., liquidation).
- Testing upgradeability against current state.

**Limitations:**
- Forks are only as accurate as the block you pinned; state may diverge if the protocol upgrades.
- Some contracts may have time-dependent logic (use `evm_setNextBlockTimestamp` to manipulate time).

---

## Chapter Summary

```
┌─────────────────────────────────────────────────────────────────┐
│                    CHAPTER 25 SUMMARY                           │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  Testing is critical due to immutability and financial risk.    │
│                                                                 │
│  Testing Pyramid:                                               │
│    • Unit tests (individual functions)                         │
│    • Integration tests (contract interactions)                 │
│    • Fuzzing (random inputs)                                   │
│    • Fork testing (live network simulation)                    │
│                                                                 │
│  Tools:                                                         │
│    • Hardhat: Mocha/Chai, ethers.js, coverage                  │
│    • Foundry: Solidity tests, fuzzing, cheatcodes              │
│    • Echidna: Advanced fuzzing                                 │
│    • solidity-coverage, forge coverage                         │
│                                                                 │
│  Best practices:                                               │
│    • Write tests before or alongside code (TDD)               │
│    • Test edge cases (zero, max, overflow)                    │
│    • Use fuzzing for invariant testing                         │
│    • Fork mainnet to test against real protocols               │
│    • Aim for high coverage but don't rely on it exclusively   │
│    • Automate tests in CI                                      │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

**Next Chapter Preview:** Chapter 26 – Smart Contract Security. We'll dive deep into common vulnerabilities (reentrancy, access control, oracle manipulation), security tools (Slither, Mythril), and the audit process.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='../6. blockchain_networks_and_ecosystems/24. other_blockchain_platforms.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='26. smart_contract_security.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
